Hotdog or Not
By: Hans, Adam, Fredrik och Robin

In [5]:
import os

extracted_files = os.listdir('hotdog-nothotdog/test/hotdog/')
print(extracted_files[0:5])


['1501.jpg', '1502.jpg', '1503.jpg', '1504.jpg', '1505.jpg']


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
)

# Set the path to the directory containing your images
path_train_hotdog = 'hotdog-nothotdog/train/'
path_test_hotdog = 'hotdog-nothotdog/test/'

train_data = datagen.flow_from_directory(
    path_train_hotdog,
    target_size = (150,150),
    class_mode = 'binary'
)

test_data = datagen.flow_from_directory(
    path_test_hotdog,
    target_size = (150,150),
    class_mode = 'binary'
)

Found 3000 images belonging to 2 classes.
Found 644 images belonging to 2 classes.


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Define the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output from the convolutional layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#
'''
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience= 5,          
    restore_best_weights=True  
)

history = model.fit(train_data, epochs=20, validation_data=test_data, callbacks=[early_stopping])
'''
#


# Train the model
history = model.fit(train_data, epochs=6, validation_data=test_data)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_data)

# Print the test accuracy
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))



Epoch 1/6
94/94 [==============================] - 24s 249ms/step - loss: 0.6878 - accuracy: 0.5890 - val_loss: 0.6323 - val_accuracy: 0.6630
Epoch 2/6
94/94 [==============================] - 24s 255ms/step - loss: 0.5874 - accuracy: 0.7010 - val_loss: 0.6174 - val_accuracy: 0.6693
Epoch 3/6
94/94 [==============================] - 24s 258ms/step - loss: 0.5683 - accuracy: 0.7027 - val_loss: 0.6443 - val_accuracy: 0.6708
Epoch 4/6
94/94 [==============================] - 23s 249ms/step - loss: 0.5443 - accuracy: 0.7260 - val_loss: 0.5820 - val_accuracy: 0.7096
Epoch 5/6
94/94 [==============================] - 23s 246ms/step - loss: 0.5112 - accuracy: 0.7520 - val_loss: 0.6132 - val_accuracy: 0.6755
Epoch 6/6
21/21 [==============================] - 2s 69ms/step - loss: 0.5700 - accuracy: 0.7252
Test accuracy: 72.52%


In [10]:
model.save('model-72.52.h5')

In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image for classification
image_path = 'hotdog-nothotdog/test/hotdog/1731.jpg'  # Replace with the actual path
img = image.load_img(image_path, target_size=(150, 150))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalize pixel values to [0, 1]

# Make a prediction using the trained model
prediction = model.predict(img)

# Interpret the prediction
if prediction < 0.5:
    print("It's a hot dog!")
else:
    print("It's not a hot dog!")
    
print(prediction)


1/1 [==============================] - 0s 86ms/step
It's a hot dog!
[[0.3781513]]


In [7]:
import tensorflow as tf

# Load the Keras model
keras_model = tf.keras.models.load_model('model-72-52.h5')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

    
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

INFO:tensorflow:Assets written to: C:\Users\fritt\AppData\Local\Temp\tmpxp_vpdu4\assets


INFO:tensorflow:Assets written to: C:\Users\fritt\AppData\Local\Temp\tmpxp_vpdu4\assets


=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #0 and tensor #4 and tensor #1 as input and produces tensor #12 as output.

Subgraph#0 main(T#0) -> [T#21]
  Op#0 CONV_2D(T#0, T#4[], T#1[]) -> [T#12]
  Op#1 MAX_POOL_2D(T#12) -> [T#13]
  Op#2 CONV_2D(T#13, T#5[], T#2[]) -> [T#14]
  Op#3 MAX_POOL_2D(T#14) -> [T#15]
  Op#4 CONV_2D(T#15, T#6[], T#3[]) -> [T#16]
  Op#5 MAX_POOL_2D(T#16) -> [T#17]
  Op#6 RESHAPE(T#17, T#9[-1, 36992]) -> [T#18]
  Op#7 FULLY_CONNECTED(T#18, T#10[], T#8[]) -> [T#19]
  Op#8 FULLY_CONNECTED(T#19, T#11[], T#7[]) -> [T#20]
  Op#9 LOGISTIC(T#20) -> [T#21]

Tensors of Subgraph#0
  T#0(serving_default_conv2d_9_input:0) shape_signature:[-1, 150, 150, 3], type:FLOAT32
  T#1(sequential_3/conv2d_9/BiasAdd/ReadVariableOp) shape:[32], type:FLOAT32 RO 128 bytes, data:[]
  T#2(sequential_3/conv2d_10/BiasAdd/ReadVariableOp) shape:[64]